Some of the data are just for 1 hour, we might want to add them too

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import re

def read_data(name = "PM10_24g"):
    root_folder = "./"
    pattern = os.path.join(root_folder, "[0-9][0-9][0-9][0-9]", f"[0-9][0-9][0-9][0-9]_{name}.xlsx")
    plik_lista = glob.glob(pattern)

    df_list = []

    for sciezka in plik_lista:
        try:
            print(f"Wczytywanie: {sciezka}")

            dopasowanie = re.search(r'(\d{4})', sciezka)
            if dopasowanie:
                rok = int(dopasowanie.group(1))
            else:
                print(f"Nie można rozpoznać roku z: {sciezka}")
                continue

            if rok <= 2015:
                skiprows = 0 
            else:
                skiprows = 1 

            df = pd.read_excel(sciezka, skiprows=skiprows, header=0, decimal=",")

            if rok <= 2015:
                df = df.drop(index=[0,1], errors="ignore")
            else:
                df = df.drop(index=[0,1,2,3], errors="ignore")

            df.rename(columns={df.columns[0]: "Data"}, inplace=True)
            df["Data"] = pd.to_datetime(df["Data"], errors='coerce')
            df.set_index("Data", inplace=True)

            df_list.append(df)

        except Exception as e:
            print(f"Błąd w pliku {sciezka}: {e}")

    df_all = pd.concat(df_list, axis=0).sort_index()
    df_all = df_all.apply(pd.to_numeric, errors='coerce')
    df_all = df_all.resample('D').mean()

    df_meta = pd.read_excel("./meta.xlsx", decimal=",")
    df_meta.set_index("Nr", inplace = True, drop = True)
    df_meta.columns = df_meta.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)
    df_meta = df_meta.rename(columns={'stary_kod_stacji_(o_ile_inny_od_aktualnego)': 'stary_kod'})
    df_meta = df_meta.rename(columns={
    'wgs84_φ_n': 'lat',
    'wgs84_λ_e': 'lon'
    })

    alias_map = {}

    for _, row in df_meta.iterrows():
        kod = row['kod_stacji']

        if pd.notna(kod):
            alias_map[kod] = kod

        if pd.notna(row['stary_kod']):
            for alias in str(row['stary_kod']).split(','):
                alias = alias.strip()
                if alias:
                    alias_map[alias] = kod

    df_renamed = df_all.copy()

    new_columns = {
        col: alias_map.get(col, col) for col in df_renamed.columns
    }

    df_renamed = df_renamed.rename(columns=new_columns)
    print(df_renamed)

    df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)
    print(df_merged)

    df_avg = df_merged.mean().reset_index()
    df_avg.columns = ['kod_stacji', 'srednia']

    df_avg = df_avg.merge(df_meta, on='kod_stacji', how='left')
    missing_coords = df_avg[df_avg[['lat', 'lon']].isna().any(axis=1)]
    print("⚠️ Stacje bez współrzędnych:")
    print(missing_coords[['kod_stacji']])
    df_avg = df_avg.dropna(subset=['lat', 'lon'])

    return df_merged, df_avg

In [2]:
import folium
from folium import plugins
from branca.colormap import linear as cm_linear

def get_map2(df_avg, name="mapa_stacje", with_heatmap=True):
    name = name + ".html"

    # Kolory wg średniej
    vmin = df_avg['srednia'].min()
    vmax = df_avg['srednia'].max()
    colormap = cm_linear.YlOrRd_09.scale(vmin, vmax)

    # Mapa
    m = folium.Map(location=[52.0, 19.0], zoom_start=6, control_scale=True)

    # Grupy typów stacji
    groups = {}
    for typ in df_avg['typ_stacji'].dropna().unique():
        groups[typ] = folium.FeatureGroup(name=f"Typ: {typ}", show=True)
        groups[typ].add_to(m)

    # Dodanie punktów
    for _, row in df_avg.iterrows():
        color = colormap(row['srednia'])
        popup_text = f"""
        <b>Kod stacji:</b> {row['kod_stacji']}<br>
        <b>Nazwa stacji:</b> {row['nazwa_stacji']}<br>
        <b>Data uruchomienia:</b> {row['data_uruchomienia']}<br>
        <b>Data zamknięcia:</b> {row['data_zamknięcia']}<br>
        <b>Typ stacji:</b> {row['typ_stacji']}<br>
        <b>Typ obszaru:</b> {row['typ_obszaru']}<br>
        <b>Rodzaj stacji:</b> {row['rodzaj_stacji']}<br>
        <b>Województwo:</b> {row['województwo']}<br>
        <b>Miejscowość:</b> {row['miejscowość']}<br>
        <b>Adres:</b> {row['adres']}<br>
        <b>Średnia:</b> {row['srednia']:.2f}
        """

        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=10,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"{row['nazwa_stacji']} ({row['kod_stacji']})<br>Średnia: {row['srednia']:.2f}",
        )

        group = groups.get(row['typ_stacji'])
        if group:
            marker.add_to(group)
        else:
            marker.add_to(m)

    # Legenda
    colormap.caption = 'Średnie zanieczyszczenie'
    colormap.add_to(m)

    # Kontrola warstw
    folium.LayerControl(collapsed=False).add_to(m)

    # Zapis
    m.save(name)
    print(f"✔️ Zapisano mapę: {name}")

In [ ]:
names = [ 
    "C6H6_1g", "CO_1g", "formaldehyd_1g", "Hg(TGM)_1g",
    "NO_1g", "O3_1g"
]

for name in names:
    print(f"\n📊 Przetwarzam: {name}")
    
    df_merged, df_avg = read_data(name)
    
    df_merged = df_merged[df_merged.index.year > 2010]
    # Zapis NetCDF
    ds = df_merged.to_xarray()
    namenc = name.replace("_1g", "_24g")
    ds.to_netcdf(f"{name}_merged.nc")

    # Generowanie mapy z ulepszeniami
    map_name = name.replace("_1g", "_24g")
    get_map2(df_avg, name=map_name)


📊 Przetwarzam: C6H6_1g
Wczytywanie: .\2003\2003_C6H6_1g.xlsx
Wczytywanie: .\2004\2004_C6H6_1g.xlsx
Wczytywanie: .\2005\2005_C6H6_1g.xlsx
Wczytywanie: .\2006\2006_C6H6_1g.xlsx
Wczytywanie: .\2007\2007_C6H6_1g.xlsx
Wczytywanie: .\2008\2008_C6H6_1g.xlsx
Wczytywanie: .\2009\2009_C6H6_1g.xlsx
Wczytywanie: .\2010\2010_C6H6_1g.xlsx
Wczytywanie: .\2011\2011_C6H6_1g.xlsx
Wczytywanie: .\2012\2012_C6H6_1g.xlsx
Wczytywanie: .\2013\2013_C6H6_1g.xlsx
Wczytywanie: .\2014\2014_C6H6_1g.xlsx


c:\Users\szymo\anaconda3\envs\geo\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Wczytywanie: .\2015\2015_C6H6_1g.xlsx
Wczytywanie: .\2016\2016_C6H6_1g.xlsx
Wczytywanie: .\2017\2017_C6H6_1g.xlsx
Wczytywanie: .\2018\2018_C6H6_1g.xlsx
Wczytywanie: .\2019\2019_C6H6_1g.xlsx
Wczytywanie: .\2020\2020_C6H6_1g.xlsx
Wczytywanie: .\2021\2021_C6H6_1g.xlsx
Wczytywanie: .\2022\2022_C6H6_1g.xlsx
Wczytywanie: .\2023\2023_C6H6_1g.xlsx
            LdLodzZachod  MzPlockOpsis  MzPlocKroJad  MzPlockPKN1  \
Data                                                                
2003-01-01           NaN      6.210000      4.284783     5.121739   
2003-01-02           NaN      3.518182      3.202083     2.775000   
2003-01-03           NaN      4.770000      4.847826     3.660417   
2003-01-04           NaN      5.028571      3.750000     2.570833   
2003-01-05           NaN      5.420000      5.062500     7.502083   
...                  ...           ...           ...          ...   
2023-12-28           NaN           NaN           NaN          NaN   
2023-12-29           NaN           Na

C:\Users\szymo\AppData\Local\Temp\ipykernel_19824\2612424235.py:83: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: C6H6.html

📊 Przetwarzam: CO_1g
Wczytywanie: .\2003\2003_CO_1g.xlsx
Wczytywanie: .\2004\2004_CO_1g.xlsx
Wczytywanie: .\2005\2005_CO_1g.xlsx
Wczytywanie: .\2006\2006_CO_1g.xlsx
Wczytywanie: .\2007\2007_CO_1g.xlsx
Wczytywanie: .\2008\2008_CO_1g.xlsx
Wczytywanie: .\2009\2009_CO_1g.xlsx
Wczytywanie: .\2010\2010_CO_1g.xlsx
Wczytywanie: .\2011\2011_CO_1g.xlsx
Wczytywanie: .\2012\2012_CO_1g.xlsx
Wczytywanie: .\2013\2013_CO_1g.xlsx
Wczytywanie: .\2014\2014_CO_1g.xlsx


...

✔️ Zapisano mapę: NO.html

📊 Przetwarzam: O3_1g
Wczytywanie: .\2000\2000_O3_1g.xlsx
Wczytywanie: .\2001\2001_O3_1g.xlsx
Wczytywanie: .\2002\2002_O3_1g.xlsx
Wczytywanie: .\2003\2003_O3_1g.xlsx
Wczytywanie: .\2004\2004_O3_1g.xlsx
Wczytywanie: .\2005\2005_O3_1g.xlsx
Wczytywanie: .\2006\2006_O3_1g.xlsx
Wczytywanie: .\2007\2007_O3_1g.xlsx
Wczytywanie: .\2008\2008_O3_1g.xlsx
Wczytywanie: .\2009\2009_O3_1g.xlsx
Wczytywanie: .\2010\2010_O3_1g.xlsx
Wczytywanie: .\2011\2011_O3_1g.xlsx
Wczytywanie: .\2012\2012_O3_1g.xlsx
Wczytywanie: .\2013\2013_O3_1g.xlsx
Wczytywanie: .\2014\2014_O3_1g.xlsx


c:\Users\szymo\anaconda3\envs\geo\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Wczytywanie: .\2015\2015_O3_1g.xlsx
Wczytywanie: .\2016\2016_O3_1g.xlsx
Wczytywanie: .\2017\2017_O3_1g.xlsx
Wczytywanie: .\2018\2018_O3_1g.xlsx
Wczytywanie: .\2019\2019_O3_1g.xlsx
Wczytywanie: .\2020\2020_O3_1g.xlsx
Wczytywanie: .\2021\2021_O3_1g.xlsx
Wczytywanie: .\2022\2022_O3_1g.xlsx
Wczytywanie: .\2023\2023_O3_1g.xlsx
            DsCzerStraza   DsJelw05  DsSniezkaObs  LbJarczWolaM  MzBelsIGFPAN  \
Data                                                                            
2000-01-01     36.136364  28.454545     59.181818     27.181818     21.583333   
2000-01-02     33.708333  33.541667     60.833333     29.083333     26.625000   
2000-01-03     51.541667  35.818182     77.041667     29.541667     24.208333   
2000-01-04     44.750000        NaN     68.916667     33.166667     24.833333   
2000-01-05     42.583333  43.625000     65.875000     39.375000     35.958333   
...                  ...        ...           ...           ...           ...   
2023-12-28           NaN    

C:\Users\szymo\AppData\Local\Temp\ipykernel_19824\2612424235.py:83: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: O3.html
